In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
titles_df = pd.read_csv("review_helper.csv")
titles_df.head()

,Unnamed: 0,Name,Platform,name_id,platform_id
0,0,0,0,Wii Sports,Wii
1,2,1,0,Mario Kart Wii,Wii
2,3,2,0,Wii Sports Resort,Wii
3,6,3,1,New Super Mario Bros.,DS
4,7,4,0,Wii Play,Wii


In [4]:
def urlmaker(platform, name):
    platform_dict = {
        "ds": "ds",
        "ps2":"playstation-2",
        "ps3": "playstation-3",
        "wii": "wii",
        "x360": "xbox-360",
        "psp": "psp",
        "ps": "playstation",
        "pc": "pc",
        "xb": "xbox",
        "gba": "game-boy-advance",
        "gc": "gamecube",
        "3ds": "3ds",
        "psv": "playstation-vita",
        "ps4": "playstation-4",
        "n64": "nintendo-64",
        "xone": "xbox-one",
        "wiiu": "wii-u",
        "dc": "dreamcast",
    }
    
    platform_fixed = platform_dict[platform.lower()]
    
    name_fixed = name.replace(" ", "-")
    name_fixed = name_fixed.replace(":", "-")
    name_fixed = name_fixed.replace("'", "")
    name_fixed = name_fixed.lower()
    
    return f"http://metacritic.com/game/{platform_fixed}/{name_fixed}"

In [11]:
game_list = []
for row in titles_df.iterrows():
    gdata = {
        "Url": urlmaker(row[1]["platform_id"], row[1]["name_id"]),
        "Name": row[1]["name_id"],
        "Platform": row[1]["platform_id"],
        "name_id": row[1]["Name"],
        "platform_id": row[1]["Platform"]
    }
    game_list.append(gdata)

In [12]:
pd.DataFrame(game_list)

,Url,Name,Platform,name_id,platform_id
0,http://metacritic.com/game/wii/wii-sports,Wii Sports,Wii,0,0
1,http://metacritic.com/game/wii/mario-kart-wii,Mario Kart Wii,Wii,1,0
2,http://metacritic.com/game/wii/wii-sports-resort,Wii Sports Resort,Wii,2,0
3,http://metacritic.com/game/ds/new-super-mario-...,New Super Mario Bros.,DS,3,1
4,http://metacritic.com/game/wii/wii-play,Wii Play,Wii,4,0
...,...,...,...,...,...
15794,http://metacritic.com/game/game-boy-advance/wo...,Woody Woodpecker in Crazy Castle 5,GBA,10813,5
15795,http://metacritic.com/game/gamecube/men-in-bla...,Men in Black II: Alien Escape,GC,2637,14
15796,http://metacritic.com/game/playstation-2/score...,SCORE International Baja 1000: The Official Game,PS2,6110,4
15797,http://metacritic.com/game/ds/know-how-2,Know How 2,DS,10814,1


In [13]:
rdf_list = []
for game in game_list:
    try:
        req = requests.request("GET", game["Url"], headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(req.text, "lxml")
        
        reviews = soup.find_all("li", class_="critic_review")
        
        for review in reviews:
            
            source = review.find("div", class_="source").text
            score = review.find("div", class_="review_grade").text.strip()
            body =  review.find("div", class_="review_body").text.strip()
            
            if (source and score and body):
                print("---------------------")
                print(game["Name"])
                print(game["Platform"])
                print(source)
                print(score)
                
            review_data = {
                "Name": game["Name"],
                "Platform": game["Platform"],
                "Name_id": game["name_id"],
                "Platform_id": game["platform_id"],
                "Review Source": source,
                "Review Score": score,
                "Review Body": body
            }
            
            rdf_list.append(review_data)
    except Exception as e:
        print(e)

---------------------
Wii Sports
Wii
Official Nintendo Magazine UK
90
---------------------
Wii Sports
Wii
GameZone
85
---------------------
Wii Sports
Wii
GameDaily
80
---------------------
Wii Sports
Wii
GamingExcellence
77
---------------------
Wii Sports
Wii
My Gamer
74
---------------------
Wii Sports
Wii
Edge Magazine
70
---------------------
Wii Sports
Wii
Digital Entertainment News
50
---------------------
Mario Kart Wii
Wii
AceGamez
100
---------------------
Mario Kart Wii
Wii
GameSpy
90
---------------------
Mario Kart Wii
Wii
Play.tm
85
---------------------
Mario Kart Wii
Wii
GameFocus
83
---------------------
Mario Kart Wii
Wii
Gamer.nl
80
---------------------
Mario Kart Wii
Wii
Deeko
80
---------------------
Mario Kart Wii
Wii
Game Chronicles
42
---------------------
Wii Sports Resort
Wii
Giant Bomb
100
---------------------
Wii Sports Resort
Wii
Cubed3
90
---------------------
Wii Sports Resort
Wii
Play.tm
87
---------------------
Wii Sports Resort
Wii
Gaming Age
83
---

---------------------
Grand Theft Auto IV
PS3
Jolt Online Gaming UK
100
---------------------
Grand Theft Auto IV
PS3
ZTGD
100
---------------------
Grand Theft Auto IV
PS3
GameSpot
100
---------------------
Grand Theft Auto IV
PS3
Gaming Age
100
---------------------
Grand Theft Auto IV
PS3
Extreme Gamer
98
---------------------
Grand Theft Auto IV
PS3
PSM3 Magazine UK
96
---------------------
Grand Theft Auto IV
PS3
GamerNode
87
---------------------
Just Dance 3
Wii
Nintendo Life
90
---------------------
Just Dance 3
Wii
Gamer.nl
80
---------------------
Just Dance 3
Wii
Switch Brasil
80
---------------------
Just Dance 3
Wii
GameShark
75
---------------------
Just Dance 3
Wii
XGN
75
---------------------
Just Dance 3
Wii
Cheat Code Central
74
---------------------
Just Dance 3
Wii
Gamekult
60
---------------------
Mario Kart 64
N64
GamePro
100
---------------------
Mario Kart 64
N64
Electronic Gaming Monthly
93
---------------------
Mario Kart 64
N64
Nintendojo
90
-----------------

---------------------
Battlefield 3
PS3
Game Informer
93
---------------------
Battlefield 3
PS3
Eurogamer Sweden
90
---------------------
Battlefield 3
PS3
Official PlayStation Magazine Benelux
90
---------------------
Battlefield 3
PS3
PSFocus
86
---------------------
Battlefield 3
PS3
PlayStation Universe
85
---------------------
Battlefield 3
PS3
Playstation: The Official Magazine (US)
80
---------------------
Battlefield 3
PS3
GamePro
70
---------------------
Need for Speed Underground
PS2
GamePro
100
---------------------
Need for Speed Underground
PS2
Warcry
91
---------------------
Need for Speed Underground
PS2
Yahoo!
90
---------------------
Need for Speed Underground
PS2
Weekly Famitsu
85
---------------------
Need for Speed Underground
PS2
Games Radar
82
---------------------
Need for Speed Underground
PS2
G4 TV
80
---------------------
Need for Speed Underground
PS2
games(TM)
50
---------------------
Tekken 3
PS
GamePro
100
---------------------
Tekken 3
PS
Electric Playgr

---------------------
Gears of War 3
X360
The A.V. Club
100
---------------------
Gears of War 3
X360
GameFocus
97
---------------------
Gears of War 3
X360
Meristation
95
---------------------
Gears of War 3
X360
VideoGamer
90
---------------------
Gears of War 3
X360
CD-Action
90
---------------------
Gears of War 3
X360
BigPond GameArena
90
---------------------
Gears of War 3
X360
GameCritics
65
---------------------
Gears of War
X360
1UP
100
---------------------
Gears of War
X360
Thunderbolt
100
---------------------
Gears of War
X360
WHAM! Gaming
97
---------------------
Gears of War
X360
ActionTrip
95
---------------------
Gears of War
X360
IC-Games
93
---------------------
Gears of War
X360
Official Xbox Magazine UK
90
---------------------
Gears of War
X360
GameCritics
80
---------------------
Metal Gear Solid
PS
Armchair Empire
100
---------------------
Metal Gear Solid
PS
Total Video Games
100
---------------------
Metal Gear Solid
PS
GMR Magazine
100
---------------------


---------------------
Donkey Kong 64
N64
GamePro
100
---------------------
Donkey Kong 64
N64
Games Domain
100
---------------------
Donkey Kong 64
N64
Gaming Maxx
98
---------------------
Donkey Kong 64
N64
Game Critics
90
---------------------
Donkey Kong 64
N64
GameSpot
90
---------------------
Donkey Kong 64
N64
GameShark
90
---------------------
Donkey Kong 64
N64
Hot Games
80
---------------------
Assassin's Creed II
X360
GamePro
100
---------------------
Assassin's Creed II
X360
GamingExcellence
97
---------------------
Assassin's Creed II
X360
GameWatcher
95
---------------------
Assassin's Creed II
X360
HellBored
92
---------------------
Assassin's Creed II
X360
Total Video Games
90
---------------------
Assassin's Creed II
X360
Thunderbolt
90
---------------------
Assassin's Creed II
X360
Destructoid
45
---------------------
Professor Layton and the Curious Village
DS
GameShark
100
---------------------
Professor Layton and the Curious Village
DS
Boomtown
90
-----------------

...

---------------------
Forza Horizon 2
XOne
LEVEL (Czech Republic)
100
---------------------
Forza Horizon 2
XOne
Multiplayer.it
93
---------------------
Forza Horizon 2
XOne
COGconnected
90
---------------------
Forza Horizon 2
XOne
Gamestyle
90
---------------------
Forza Horizon 2
XOne
Digital Chumps
86
---------------------
Forza Horizon 2
XOne
Post Arcade (National Post)
85
---------------------
Forza Horizon 2
XOne
Quarter to Three
20
---------------------
Sonic Colors
DS
Switch Brasil
90
---------------------
Sonic Colors
DS
Destructoid
85
---------------------
Sonic Colors
DS
SpazioGames
85
---------------------
Sonic Colors
DS
VideoGamer
80
---------------------
Sonic Colors
DS
GameTrailers
79
---------------------
Sonic Colors
DS
Nintendo Power
75
---------------------
Sonic Colors
DS
Gamestyle
60
---------------------
Tom Clancy's Ghost Recon 2
XB
XboxAddict
96
---------------------
Tom Clancy's Ghost Recon 2
XB
Gamenikki
90
---------------------
Tom Clancy's Ghost Recon 2
XB

---------------------
Need for Speed Carbon
PS2
Official Playstation 2 Magazine UK
90
---------------------
Need for Speed Carbon
PS2
PSM3 Magazine UK
83
---------------------
Need for Speed Carbon
PS2
Games Master UK
79
---------------------
Need for Speed Carbon
PS2
GameSpot
74
---------------------
Need for Speed Carbon
PS2
PSW Magazine UK
70
---------------------
Need for Speed Carbon
PS2
VideoGamer
70
---------------------
Need for Speed Carbon
PS2
Game Revolution
42
---------------------
MVP Baseball 2004
PS2
GamePro
100
---------------------
MVP Baseball 2004
PS2
XGP Gaming
93
---------------------
MVP Baseball 2004
PS2
Game Revolution
91
---------------------
MVP Baseball 2004
PS2
Yahoo!
90
---------------------
MVP Baseball 2004
PS2
1UP
90
---------------------
MVP Baseball 2004
PS2
Sports Gaming Network
88
---------------------
MVP Baseball 2004
PS2
netjak
70
---------------------
WWE SmackDown vs Raw 2008
PS3
PSM Magazine
90
---------------------
WWE SmackDown vs Raw 2008
PS

---------------------
SingStar '80s
PS2
Jolt Online Gaming UK
90
---------------------
SingStar '80s
PS2
TotalPlayStation
90
---------------------
SingStar '80s
PS2
Games Radar
81
---------------------
SingStar '80s
PS2
GameSpy
80
---------------------
SingStar '80s
PS2
IGN
75
---------------------
SingStar '80s
PS2
Gaming Age
75
---------------------
SingStar '80s
PS2
GameDaily
60
---------------------
EA Playground
Wii
DailyGame
80
---------------------
EA Playground
Wii
Play.tm
75
---------------------
EA Playground
Wii
Cheat Code Central
72
---------------------
EA Playground
Wii
GameZone
68
---------------------
EA Playground
Wii
IGN
66
---------------------
EA Playground
Wii
Maxi Consolas (Portugal)
65
---------------------
EA Playground
Wii
Eurogamer
30
---------------------
EA Sports Active 2
Wii
Da Gameboyz
88
---------------------
EA Sports Active 2
Wii
NZGamer
85
---------------------
EA Sports Active 2
Wii
GamingXP
83
---------------------
EA Sports Active 2
Wii
IGN
80
----

---------------------
Battle Arena Toshinden
PS
TotalGames.net
75
---------------------
Battle Arena Toshinden
PS
All Game Guide
70
---------------------
Battle Arena Toshinden
PS
IGN
70
---------------------
Battle Arena Toshinden
PS
Thunderbolt
60


KeyboardInterrupt: 